In [22]:
import os
import pickle
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import RepeatedKFold, GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

path = input('Por favor adicione o diretorio de trabalho?')
for dirname, _, filename in os.walk(path, topdown = True):
    for filename in filename:
        if filename == 'test.csv':
                teste = pd.read_csv(os.path.join(dirname,filename))
        else:
            if filename == 'train.csv':
                treino = pd.read_csv(os.path.join(dirname,filename))
            else:
                print("Deu ruim irmão!!")

X = treino.drop(['Cover_Type','Id','Unnamed: 0'],axis=1) 
y= treino.Cover_Type
test_ids = teste['Id']

#Coleta de Dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)

Dtrain = xgb.DMatrix(X_train, label=y_train)
Dtest = xgb.DMatrix(X_test, label=y_test)
teste.drop(['Id','Unnamed: 0'],axis=1,inplace = True)
Dpred = xgb.DMatrix(teste)

Por favor adicione o diretorio de trabalho? C:\github\Kaggle_Challenges\Machine_Learning_Competitions\data


In [23]:
%%time
#XGBoost

#Treinamento
num_class = 8
objective = "multi:softmax"
predictor = 'cpu_predictor'
booster = 'gbtree'
grow_policy = 'depthwise'
max_depth = 20
gamma = 0
num_parallel_tree = 1
num_boost_round = 100
learning_rate = 0.1
colsample_bynode= 1
subsample=1
min_child_weight=1
reg_lambda = 1
alpha = 0
max_leaves = 0
process_type = 'update'
eval_metric = {'eval_metric': 'merror'}
watchlist = [(Dtest, 'eval'), (Dtrain, 'train')]


params= {'max_depth' : max_depth,
         'gamma': gamma,
         'objective': objective,
         'num_class' : num_class,
         'num_parallel_tree' : num_parallel_tree, 
         'num_boost_round' : num_boost_round,
         'learning_rate': learning_rate,
         'colsample_bynode': colsample_bynode,
         'subsample': subsample,
         'min_child_weight':min_child_weight,
         'reg_lambda':reg_lambda,
         'alpha':alpha,
         'max_leaves':max_leaves,
         'predictor' : predictor,
         'booster' : booster,
         'grow_policy' :grow_policy}


bst= xgb.cv( dtrain = Dtrain, params=params, num_boost_round=num_boost_round, nfold = 3,
                                   metrics = 'merror',
                                   obj=None,
                                   feval=None,
                                   maximize=False,
                                   as_pandas = True,
                                   #early_stopping_rounds= 3, 
                                   verbose_eval=100,
                                   callbacks=[xgb.callback.print_evaluation(show_stdv=True)])
print('\n')
print(bst.tail(15))

[0]	train-merror:0.093006+0.000927931	test-merror:0.22214+0.00691585
[0]	train-merror:0.093006+0.000927931	test-merror:0.22214+0.00691585
[1]	train-merror:0.0719247+0.00199464	test-merror:0.205771+0.00852516
[2]	train-merror:0.05754+0.00152877	test-merror:0.194196+0.00754717
[3]	train-merror:0.050719+0.0011137	test-merror:0.190724+0.00596623
[4]	train-merror:0.045015+0.00193959	test-merror:0.18626+0.00390551
[5]	train-merror:0.040468+0.00167126	test-merror:0.184772+0.00430978
[6]	train-merror:0.03559+0.000535741	test-merror:0.183036+0.00404511
[7]	train-merror:0.0314153+0.000824597	test-merror:0.178158+0.00581775
[8]	train-merror:0.02753+0.000463966	test-merror:0.176587+0.00599366
[9]	train-merror:0.0253393+0.00030931	test-merror:0.175099+0.00526524
[10]	train-merror:0.022569+0.000441319	test-merror:0.175926+0.00399593
[11]	train-merror:0.020337+0.000730092	test-merror:0.17419+0.00342643
[12]	train-merror:0.018105+0.000828731	test-merror:0.172619+0.00477467
[13]	train-merror:0.0159557+